In [4]:
import cv2
import numpy as np 
import torch 
from torchvision import transforms, models 
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision
from sklearn.metrics import pairwise_distances
from sklearn.decomposition import LatentDirichletAllocation 
from scipy.spatial import distance
from torchvision import transforms

In [28]:
from sklearn.tree import DecisionTreeClassifier

In [7]:
from torch.utils.data import Subset

In [9]:
from tqdm import tqdm

In [21]:
import utils

In [6]:
dataset = torchvision.datasets.Caltech101('/Users/niteeqsheik/Desktop/Phase3',download=True)
data_loader = torch.utils.data.DataLoader(dataset,
batch_size=4,
shuffle=True,
num_workers=8)
print(len(dataset))

131740031it [00:04, 26916279.73it/s]


Extracting /Users/niteeqsheik/Desktop/Phase3/caltech101/101_ObjectCategories.tar.gz to /Users/niteeqsheik/Desktop/Phase3/caltech101


14028800it [00:00, 18831818.28it/s]


Extracting /Users/niteeqsheik/Desktop/Phase3/caltech101/Annotations.tar to /Users/niteeqsheik/Desktop/Phase3/caltech101
8677


In [8]:
even_indices = [i for i in range(0, len(dataset), 2)]
even_dataset = Subset(dataset, even_indices)
even_data_loader = torch.utils.data.DataLoader(even_dataset, batch_size=4, shuffle=True, num_workers=8)
print(len(even_dataset))

4339


In [11]:
resnet50 = models.resnet50(pretrained = 50)

In [12]:
resnet50.eval() 

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [16]:
transform = transforms.Compose([
    transforms.Resize(256),             # Resize the image to 256x256 pixels
    transforms.CenterCrop(224),         # Crop the image to 224x224 pixels about the center
    transforms.Grayscale(num_output_channels=3),  
    transforms.ToTensor(),              # Convert the image to a tensor
    transforms.Normalize(mean=[0.53994344, 0.52009986, 0.49254049], std=[0.31415099, 0.30712622, 0.31878401]),  # Normalize the image
])
def extract_features_individual(even_dataset, model, transform):
    features = []
    labels = []
    with torch.no_grad():
        for image, label in tqdm(even_dataset, desc="Extracting Features"):
            image = transform(image).unsqueeze(0)  # Apply transform and add batch dimension
            output = model(image)
            features.append(output.squeeze(0)) 
            labels.append(label) # Remove batch dimension
    return features,labels
# Extract Features from Even-Numbered Images
even_features, even_labels = extract_features_individual(even_dataset, resnet50, transform)

Extracting Features: 100%|██████████| 4339/4339 [02:53<00:00, 25.08it/s]


In [20]:
# import matplotlib.pyplot as plt
# from torchvision import datasets, transforms
# from torch.utils.data import Subset
# import numpy as np

# # Assuming you have already defined and loaded your dataset and even_dataset

# # Function to show an image
# def imshow(img):
#     img = img / 2 + 0.5  # unnormalize
#     npimg = img.numpy()
#     plt.imshow(np.transpose(npimg, (1, 2, 0)))
#     plt.show()

# # Display three images for each label
# unique_labels = set(even_labels)

# for label in unique_labels:
#     print(f"Label: {label}")
#     count = 0
#     for i, (image, lbl) in enumerate(even_dataset):
#         if lbl == label and count < 3:
#             imshow(transforms.ToTensor()(image))
#             count += 1
#         if count >= 3:
#             break

### KNN Classifier for even numbered images

In [26]:
from sklearn.neighbors import KNeighborsClassifier
import numpy as np 
def prepare_knn_classifier(k,features,labels):
    """
    Prepares a k-NN classifier.

    Args:
    k (int): The number of neighbors to use for k-NN.
    features (list of torch.Tensor): The list of feature tensors from the ResNet model.
    labels (list): The list of labels corresponding to the features.

    Returns:
    KNeighborsClassifier: The trained k-NN classifier
    """
    knn = KNeighborsClassifier(n_neighbors = k) 
    knn.fit(features,labels)
    return knn 
db_path = 'caltech101_features.db'
even_features_db,even_labels_db = utils.load_even_images_and_labels(db_path)

#convert the list of numpy arrays to 2D numpy arrays 
X = np.stack(even_features_db)
y = np.array(even_labels_db)
k = int(input("Enter the number of neighbors for k-NN: "))
knn_classifier = prepare_knn_classifier(k,X,y)


### Decision Tree Classifier

In [29]:
def train_decision_tree(features, labels, **dt_params): 
    """
    Trains a Decision Tree classifier using all provided features and labels.

    Args:
    features (numpy.ndarray): The feature vectors.
    labels (numpy.ndarray): The labels corresponding to the features.
    **dt_params: Additional parameters to pass to the Decision Tree classifier.

    Returns:
    DecisionTreeClassifier: The trained Decision Tree classifier.
    """
    # Initialize the Decision Tree classifier
    decision_tree = DecisionTreeClassifier(**dt_params)

    # Train the classifier
    decision_tree.fit(features, labels)

    return decision_tree
decision_tree_classifier = train_decision_tree(X, y)

### PPR CODE

In [33]:
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity

In [43]:
import numpy as np 
features_array = np.stack(even_labels_db)
print(features_array.shape)

(4339,)


In [51]:
import numpy as np

# Check if even_features is a list of NumPy arrays
if len(even_features_db) > 0 and isinstance(even_features_db[0], np.ndarray):
    print("The elements of even_features are NumPy arrays.")
else:
    print("The elements of even_features are not NumPy arrays.")


The elements of even_features are NumPy arrays.


In [50]:
# import numpy as np

# # Ensure all tensors are on the CPU and convert to NumPy arrays
# even_features_np = np.stack([tensor.cpu().numpy() for tensor in even_features_db])

# print("Combined features shape:", even_features_np.shape)


AttributeError: 'numpy.ndarray' object has no attribute 'cpu'

In [54]:
import networkx as nx
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def create_similarity_graph(features):
    """
    Creates an undirected graph based on cosine similarity between features.

    Args:
    features (numpy.ndarray): Feature vectors of the images.

    Returns:
    networkx.Graph: An undirected graph representing the similarities between images.
    """
    similarity_matrix = cosine_similarity(features)
    G = nx.Graph()
    for i in range(similarity_matrix.shape[0]):
        for j in range(similarity_matrix.shape[1]):
            if i != j:  # Avoid self-loops
                G.add_edge(i, j, weight=similarity_matrix[i][j])
    return G

def personalized_pagerank_classifier(graph, labels, alpha=0.85, max_iter=500):
    unique_labels = set(labels)
    predictions = {}

    for node in graph.nodes():
        pagerank_scores = {}
        for label in unique_labels:
            personalization = {i: 0.1 if labels[i] != label else 1 for i in graph.nodes()}  # Softer personalization
            try:
                pagerank = nx.pagerank(graph, alpha=alpha, personalization=personalization, max_iter=max_iter)
                pagerank_scores[label] = pagerank[node]
                print(f"Success: PageRank converged for label '{label}' at node {node}.")
            except nx.PowerIterationFailedConvergence as e:
                print(f"Error: PageRank failed to converge for label '{label}' at node {node}. {e}")
                pagerank_scores[label] = 0  # Handle non-convergence

        predicted_label = max(pagerank_scores, key=pagerank_scores.get)
        predictions[node] = predicted_label

    return predictions

# Load your features and labels
# even_features, even_labels = ...

# Create a graph based on the features
G = create_similarity_graph(np.stack(even_features_db))

# Classify the images
predictions = personalized_pagerank_classifier(G, even_labels_db)

# Now, `predictions` contains the predicted labels for each image


Error: PageRank failed to converge for label '14' at node 0. (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 500 iterations')
Error: PageRank failed to converge for label '92' at node 0. (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 500 iterations')
Error: PageRank failed to converge for label '95' at node 0. (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 500 iterations')
Error: PageRank failed to converge for label '86' at node 0. (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 500 iterations')
Error: PageRank failed to converge for label '15' at node 0. (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 500 iterations')
Error: PageRank failed to converge for label '23' at node 0. (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 500 iterations')
Error: PageRank failed to converge for l

In [ ]:
orint